In [1]:
pip -q install datasets transformers evaluate huggingface_hub bitsandbytes

In [ ]:
!pip -q install --upgrade transformers

In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np
from transformers import DataCollatorWithPadding

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [5]:
dataset_dict = load_dataset("shawhin/phishing-site-classification")
dataset_dict

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/98.0k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/21.4k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/24.5k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2100 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/450 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/450 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 2100
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 450
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 450
    })
})

In [13]:
model_path = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_path)
id2label = {0: "Safe", 1: "Not Safe"}
label2id = {"Safe": 0, "Not Safe": 1}
model = AutoModelForSequenceClassification.from_pretrained(model_path,
                                                           num_labels=2,
                                                           id2label=id2label,
                                                           label2id=label2id,)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
for name, param in model.base_model.named_parameters():
  if "pooler" in name:
        param.requires_grad = True
  else:
        param.requires_grad = False

In [15]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [16]:
def preprocess_function(examples):
    return tokenizer(examples["text"], padding='max_length', truncation=True)
tokenized_dataset = dataset_dict.map(preprocess_function, batched=True)
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/2100 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

In [17]:
accuracy = evaluate.load("accuracy")
auc_score = evaluate.load("roc_auc")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    probabilities = np.exp(predictions) / np.exp(predictions).sum(-1, keepdims=True)
    positive_class_probs = probabilities[:, 1]
    auc = np.round(auc_score.compute(prediction_scores=positive_class_probs, references=labels)['roc_auc'],3)

    predicted_classes = np.argmax(predictions, axis=1)
    acc = np.round(accuracy.compute(predictions=predicted_classes, references=labels)['accuracy'],3)

    return {"Accuracy": acc, "AUC": auc}

In [18]:
lr = 2e-4
batch_size = 32
num_epochs = 10

training_args = TrainingArguments(
    output_dir="bert-phishing-classifier_teacher",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

<ipython-input-21-ea83d2950a86>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [22]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Auc
1,0.394200,0.378818,0.844000,0.915000
2,0.359000,0.350077,0.860000,0.926000
3,0.333800,0.348449,0.862000,0.932000
4,0.318300,0.337592,0.871000,0.934000
5,0.298000,0.325172,0.880000,0.936000
6,0.298000,0.321483,0.876000,0.937000
7,0.297000,0.323359,0.876000,0.937000
8,0.291800,0.318797,0.882000,0.938000
9,0.286200,0.319368,0.871000,0.938000
10,0.282300,0.317861,0.878000,0.938000


TrainOutput(global_step=660, training_loss=0.3158749146894975, metrics={'train_runtime': 444.5505, 'train_samples_per_second': 47.239, 'train_steps_per_second': 1.485, 'total_flos': 2781815371776000.0, 'train_loss': 0.3158749146894975, 'epoch': 10.0})

In [23]:
predictions = trainer.predict(tokenized_dataset["test"])
logits = predictions.predictions
labels = predictions.label_ids

metrics = compute_metrics((logits, labels))
print(metrics)

{'Accuracy': np.float64(0.871), 'AUC': np.float64(0.947)}


In [24]:
from huggingface_hub import notebook_login
notebook_login()

In [25]:
trainer.push_to_hub('majorSeaweed/distilled_Bert_Uncased_FishingURL_Student_Finetuned')
tokenizer.push_to_hub('majorSeaweed/distilled_Bert_Uncased_FishingURL_Student_Finetuned')

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

events.out.tfevents.1742373344.2097af05189c.2406.0:   0%|          | 0.00/5.13k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

events.out.tfevents.1742373391.2097af05189c.2406.1:   0%|          | 0.00/11.3k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/majorSeaweed/distilled_Bert_Uncased_FishingURL_Student_Finetuned/commit/f38ac0c2328243a51208bf145f3f0721b43d4d78', commit_message='Upload tokenizer', commit_description='', oid='f38ac0c2328243a51208bf145f3f0721b43d4d78', pr_url=None, repo_url=RepoUrl('https://huggingface.co/majorSeaweed/distilled_Bert_Uncased_FishingURL_Student_Finetuned', endpoint='https://huggingface.co', repo_type='model', repo_id='majorSeaweed/distilled_Bert_Uncased_FishingURL_Student_Finetuned'), pr_revision=None, pr_num=None)